In [1]:
import sys
if ".." not in sys.path:
    sys.path.append("..")

import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import wandb
import functools

from src.display import showarray
from src.mask import MaskGenerator
from src.datagen import DatasetFillGenerator
from src.augmenters import identical_augmenter, mask_image_augmenter
from src.builders.unet import UNETBuilder

In [2]:
config = {
  "learning_rate": 0.001,
  "epochs": 10,
  "batch_size": 16,
  "mask_gen_degree": "MEDIUM",
  "mask_gen_min_width": 3,
  "mask_gen_max_width": 9,
  "unet_n_filters": 32,
  "unet_n_layers": 2,
  "unet_n_convs": 1, 
  "unet_activation": "elu",
  "learning_rate": 0.001
}
wandb.init(project="cv3-ii-ae-unet", entity="put_dl_team", config=config)
# wandb.config.update(config)


IMAGE_SIZE = (256, 256)
CHANNELS = 3
BATCH_SIZE = wandb.config["batch_size"]

MASK_GEN_PARAM = {
    "degree": wandb.config["mask_gen_degree"],
    "min_width": wandb.config["mask_gen_min_width"],
    "max_width": wandb.config["mask_gen_max_width"],
}

mask_generator = MaskGenerator(*IMAGE_SIZE, CHANNELS, **MASK_GEN_PARAM)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: sbartekt (put_dl_team). Use `wandb login --relogin` to force relogin


In [3]:
def scale(x):
    return x / 255.0

def recast_to_image(tensor: tf.Tensor) -> np.ndarray:
    return tf.cast(tensor * 255, tf.uint8).numpy()


ds_train, ds_valid = tf.keras.preprocessing.image_dataset_from_directory(
    directory="../data/1-8size", label_mode=None, image_size=IMAGE_SIZE, batch_size=BATCH_SIZE,
    shuffle=True, seed=42, validation_split=0.3, subset="both",
)
ds_train = ds_train.map(scale)
ds_valid = ds_valid.map(scale)

for batch in ds_valid.take(1):
    showcase_images = batch[:5]


class ImageFillCallback(tf.keras.callbacks.Callback):
    def __init__(self, model, showcase_images, mask_generator):
        self.model = model
        self.showcase_images = showcase_images
        self.masked_images = mask_image_augmenter(self.showcase_images, mask_generator, 1.0)

    def on_epoch_end(self, epoch, logs=None):
        nn_filled = self.model.predict(self.masked_images)
        all_joint = []
        for i in range(5):
            masked = recast_to_image(self.masked_images[i])
            original_image = recast_to_image(self.showcase_images[i])
            filled_image = recast_to_image(nn_filled[i])
            joint = np.concatenate([masked, original_image, filled_image], axis=1)
            all_joint.append(joint)
        all_joint = np.concatenate(all_joint, axis=0)
        wandb.log({"sample_fill": wandb.Image(all_joint)})

dataset_image_augmenter = functools.partial(mask_image_augmenter, mask_generator=mask_generator, max_val=1.0)

np.random.seed(42)
train_generator = DatasetFillGenerator(ds_train, IMAGE_SIZE, CHANNELS, dataset_image_augmenter)
valid_generator = DatasetFillGenerator(ds_valid, IMAGE_SIZE, CHANNELS, dataset_image_augmenter)

Found 2188 files belonging to 1 classes.
Using 1532 files for training.
Using 656 files for validation.


In [4]:
IM_SHAPE = (*IMAGE_SIZE, CHANNELS)
builder = UNETBuilder(IM_SHAPE, IM_SHAPE, n_filters=wandb.config["unet_n_filters"], n_layers=wandb.config["unet_n_layers"], n_convs=wandb.config["unet_n_convs"], activation=wandb.config["unet_activation"])
model = builder.build()
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 256, 256, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 256, 256, 32  896         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 max_pooling2d (MaxPooling2D)   (None, 128, 128, 32  0           ['conv2d[0][0]']                 
                                )                                                             

In [5]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=wandb.config["learning_rate"]),
    loss=tf.keras.losses.MeanSquaredError()
)

In [6]:
model.fit(train_generator, epochs=10, validation_data=valid_generator, callbacks=[wandb.keras.WandbCallback(), ImageFillCallback(model, showcase_images, mask_generator)])

wandb: WARNING The save_model argument by default saves the model in the HDF5 format that cannot save custom objects like subclassed models and custom layers. This behavior will be deprecated in a future release in favor of the SavedModel format. Meanwhile, the HDF5 model is saved as W&B files and the SavedModel as W&B Artifacts.


Epoch 1/10
96/96 [==============================] - ETA: 0s - loss: 0.0248

INFO:tensorflow:Assets written to: c:\Users\sbart\OneDrive - put.poznan.pl\Semestr V\Computer Vision\Project 3\notebooks\wandb\run-20230119_164814-k9gkstdu\files\model-best\assets


INFO:tensorflow:Assets written to: c:\Users\sbart\OneDrive - put.poznan.pl\Semestr V\Computer Vision\Project 3\notebooks\wandb\run-20230119_164814-k9gkstdu\files\model-best\assets
wandb: Adding directory to artifact (c:\Users\sbart\OneDrive - put.poznan.pl\Semestr V\Computer Vision\Project 3\notebooks\wandb\run-20230119_164814-k9gkstdu\files\model-best)... Done. 0.0s


96/96 [==============================] - 445s 5s/step - loss: 0.0248 - val_loss: 0.0133
Epoch 2/10
96/96 [==============================] - ETA: 0s - loss: 0.0095

...

INFO:tensorflow:Assets written to: c:\Users\sbart\OneDrive - put.poznan.pl\Semestr V\Computer Vision\Project 3\notebooks\wandb\run-20230119_164814-k9gkstdu\files\model-best\assets
wandb: Adding directory to artifact (c:\Users\sbart\OneDrive - put.poznan.pl\Semestr V\Computer Vision\Project 3\notebooks\wandb\run-20230119_164814-k9gkstdu\files\model-best)... Done. 0.0s


96/96 [==============================] - 510s 5s/step - loss: 0.0024 - val_loss: 0.0022
Epoch 9/10
96/96 [==============================] - 505s 5s/step - loss: 0.0022 - val_loss: 0.0023
Epoch 10/10
96/96 [==============================] - ETA: 0s - loss: 0.0023

INFO:tensorflow:Assets written to: c:\Users\sbart\OneDrive - put.poznan.pl\Semestr V\Computer Vision\Project 3\notebooks\wandb\run-20230119_164814-k9gkstdu\files\model-best\assets


INFO:tensorflow:Assets written to: c:\Users\sbart\OneDrive - put.poznan.pl\Semestr V\Computer Vision\Project 3\notebooks\wandb\run-20230119_164814-k9gkstdu\files\model-best\assets
wandb: Adding directory to artifact (c:\Users\sbart\OneDrive - put.poznan.pl\Semestr V\Computer Vision\Project 3\notebooks\wandb\run-20230119_164814-k9gkstdu\files\model-best)... Done. 0.1s


96/96 [==============================] - 498s 5s/step - loss: 0.0023 - val_loss: 0.0020


In [7]:
wandb.finish(0)

epoch,▁▂▃▃▄▅▆▆▇█
loss,█▃▂▁▁▁▁▁▁▁
val_loss,█▃▂▂▁▁▁▁▁▁
best_epoch,9
best_val_loss,0.00199
epoch,9
loss,0.00226
val_loss,0.00199
